# Büyük Veri Final Projesi

** Kütüphaneler import edilir

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")
import seaborn as sns
from datetime import datetime

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

** Gerekli veri kümeleri eklenir

In [ ]:
item_categ = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_sub = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

* info() methodu ile, girdi sayısı, her bir sütundaki null olmayan girdi sayısı ve sütunun veri tipi bilgilerine erişilir.

* head() methodu ile, veri kümesi içeriği görüntülenme sağlanır.

In [ ]:
sample_sub.info()
sample_sub.head(10)

In [ ]:
test.info()
test.head()

* value_counts() methodu, sütundaki NaN olmayan her bir unique değerin kaç kez kullanıldığını gösteren bir seri döndürür

* shape ise bu seri dizisinin şeklini döndürür

In [ ]:
magaza_id=test["shop_id"].value_counts().shape  #kaç farklı mağaza id'si olduğunu gösterir
print("Mağaza İd",magaza_id)
urun_id=test["item_id"].value_counts().shape  #kaç farklı mağaza id'si olduğunu gösterir
print("Ürün İd ",urun_id)

In [ ]:
plt.plot(test.shop_id,test.item_id) 

plt.title("shop_id - item_id Grafiği")

plt.xlabel("shop_id")

plt.ylabel("item_id")
plt.show()

In [ ]:
items.info()
items.head()

In [ ]:
urun_adi=items["item_name"].value_counts().shape  #kaç farklı ürün çeşidi olduğunu gösterir
print("Ürün Çeşidi",urun_adi)
urun_kategori=items["item_category_id"].value_counts().shape  #kaç farklı ürün kategorisi olduğunu gösterir
print("Ürün Kategori Sayısı",urun_kategori)

In [ ]:
item_categ.info()
item_categ.head()

In [ ]:
shops.info()
shops.head()

In [ ]:
sales_train.info()
sales_train.head()

** sales_train veri kümesi içindeki date değişkeninde belirtilen tarihleri ay ve yıla göre sınıflandırma işlemi yapılır

In [ ]:
sales_train['year'] = pd.to_datetime(sales_train['date']).dt.strftime('%Y')
sales_train['month'] = sales_train.date.apply(lambda x: datetime.strptime(x,'%d.%m.%Y').strftime('%m')) 
sales_train.head()

* value_counts().shape methodu ile kaç farklı yıla ait verilerin olduğu döndürülür

* unique() methodu ile benzersiz olanlar dizisi görüntülenir

In [ ]:
yil=sales_train["year"].value_counts().shape  
print("yil",yil)
yil = sales_train['year'].unique() 
yil

* Yıllara ait dağılım grafiği gösterilir

In [ ]:
yil1=sales_train[sales_train["year"]=="2013"]["item_id"] #red
yil2=sales_train[sales_train["year"]=="2014"]["item_id"] #green
yil3=sales_train[sales_train["year"]=="2015"]["item_id"] #blue

plt.figure(figsize=(15,5))
sns.distplot(yil1,color="red") #dağılım grafiği
sns.distplot(yil2,color="green")
sns.distplot(yil3,color="blue")

** new_data adında bir nesne oluşturulur ve sales_train veri kümesi değişkenleri yeni oluşturulan nesneye atanır

*pivot_table() verileri gruplandırmak için kullanılır*

*>index: gruplama ölçütleri*

*>values: toplanacak sütun*
      
*>columns: gruplayıcı*   

*>fill_value: eksik değerlere 0 atar*

*>aggfunc: işlev*

*>reset.index: data.frame dizinini sıfırlama*

In [ ]:
new_data = sales_train.copy()

new_data = new_data.pivot_table(index=['shop_id', 'item_id'],values=['item_cnt_day'],columns=['date_block_num'],  fill_value=0, aggfunc='sum').reset_index()

new_data.head()

** test datası ile new_data merge() methodu ile  birleştirilir ve düzenlenilir

In [ ]:
new_data = pd.merge(test,new_data,on = ['item_id','shop_id'],how = 'left')
new_data.head()

* fillna() methodu ile tüm NaN değerleri 0 ile doldurulur

In [ ]:
new_data.fillna(0,inplace = True)
new_data.head()

* drop() methodu ile istenilen etiketler satır ve sütunlardan çıkarılır

In [ ]:
new_data.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
new_data.head() 

** Veri kümesi eğitim ve test veri kümeleri olarak ayrılır 

In [ ]:
# X eğitim veri kümesini new_datanın son sütunu olarak alıyoruz
X_train = np.expand_dims(new_data.values[:,:-1],axis = 2)
# son sütun etiketimiz
y_train = new_data.values[:,-1:]
# X test veri kümesi için new_datanın ilk sütunu olarak alıyoruz
X_test = np.expand_dims(new_data.values[:,1:],axis = 2)

print(X_train.shape,y_train.shape,X_test.shape)

** Veri kümesi eğitimi için Sequential adında bir nesne oluştularak RNN başlatılır ve katmanlar eklenir son olarak derleme yapılır

In [ ]:
regressor = Sequential()

regressor.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 64))
regressor.add(Dropout(0.2))

regressor.add(Dense(units =1))

regressor.compile(optimizer = 'adam', loss ='mean_squared_error', metrics=['accuracy'])

regressor.summary()

** training_model adında bir nesne oluşturulur ve oluşturulan bu nesne modeli SimpleRnn ile eğitilir

In [ ]:
training_model=regressor.fit(X_train, y_train, validation_split=0.3, epochs = 5, batch_size = 4096)

In [ ]:
y_predict = regressor.predict(X_test)

** GridSearch ile en iyi çalışan parametreler bulunur

In [ ]:
def create_hyperparams_grid(optimizer):
    grid_model = Sequential()
    grid_model.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    grid_model.add(Dropout(0.2))
    grid_model.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True))
    grid_model.add(Dropout(0.2))
    grid_model.add(SimpleRNN(units= 64, activation='relu', use_bias= True, return_sequences = True))
    grid_model.add(Dropout(0.2))
    grid_model.add(SimpleRNN(units= 64))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(1))
  
    grid_model.compile(loss = 'mse',optimizer = optimizer, metrics = ['accuracy'])
    
    return regressor

grid_model = KerasClassifier(build_fn=create_hyperparams_grid)

parameters = {'batch_size' : [4000,4096],'epochs' : [5,10],'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,param_grid = parameters,scoring = 'accuracy',cv = 2)

In [ ]:
grid_search = grid_search.fit(X_train,y_train)

* En iyi parametreler seçilir

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("En iyi parametreler:", best_parameters)
print("En iyi doğruluk değeri:", best_accuracy)

** Bir sonuc adında bir değişken oluştur ve içine ID ve item_cnt_mounth sütunlarını gruplandırdığın test veri kümesinden gelen tahmin değerlerini atanır


In [ ]:
y_predict = y_predict.clip(0,20)
 
sonuc = pd.DataFrame({'ID':test['ID'],'item_cnt_month':y_predict.ravel()})
sonuc.head()

** Tahminler csv dosyasına çıktı olarak aktarılır

In [ ]:
sonuc.to_csv('sonuc_tahmin.csv',index = False)